In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import IsolationForest
import seaborn as sns
from df_functions import read_csv_file, extract_users, format_data, split_and_reformat, reqs_per_5mins, clean_reqlogs, calculate_variance, count_unique_reqs, longestConsecutive
from vectorize import vectorizer
import math

In [ ]:
#Fetch data from csv file
data = read_csv_file('requests.csv')

#Extracts a list of all unique users in dataframe
users = extract_users(data)

In [20]:
#Fetches the first user in list
user1 = users[0]

#This function will only be used for training data
def split_user_df(dataframe, user):

    #Dataframe containing all requests made by chosen user
    user_data = dataframe.loc[data['userID'] == user]
    number_of_reqs = user_data.shape[0]
    partitions = number_of_reqs/50
    partitions = math.floor(partitions)

    #Splits the data frame into smaller chunks of ~50 requests
    dfs = np.array_split(user_data, partitions)
    
    #Returns a list of dataframes
    return dfs

#Each of these dataframes represent how data fetched from redis might look like
data_chunk = split_user_df(data, user1)[0]

,timestamp,userID,sessionID,expiring,URL
0,1676373343936,user-1003,c85cbd00-d2fc-4cda-9f5c-66616847d851,1676373495816,/chat/send/442
2,1676373344456,user-1003,c85cbd00-d2fc-4cda-9f5c-66616847d851,1676373495816,/chat/send/969
26,1676373348109,user-1003,c85cbd00-d2fc-4cda-9f5c-66616847d851,1676373495816,/searchUsers?page=2036
29,1676373348626,user-1003,c85cbd00-d2fc-4cda-9f5c-66616847d851,1676373495816,/chat/send/647
31,1676373349146,user-1003,c85cbd00-d2fc-4cda-9f5c-66616847d851,1676373495816,/chat/send/886
32,1676373349666,user-1003,c85cbd00-d2fc-4cda-9f5c-66616847d851,1676373495816,/chat/send/885
35,1676373350186,user-1003,c85cbd00-d2fc-4cda-9f5c-66616847d851,1676373495816,/chat/send/866
38,1676373350706,user-1003,c85cbd00-d2fc-4cda-9f5c-66616847d851,1676373495816,/chat/send/888
44,1676373351226,user-1003,c85cbd00-d2fc-4cda-9f5c-66616847d851,1676373495816,/chat/send/267
47,1676373351746,user-1003,c85cbd00-d2fc-4cda-9f5c-66616847d851,1676373495816,/chat/send/709
